In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [26]:
df_1 = pd.read_csv(r'C:\Users\Cecilia\Downloads\ironhack\coursework\group_work\group_project_week5_6\second_project\data\clean\df_merged.csv')
df_2 = pd.read_csv(r'C:\Users\Cecilia\Downloads\ironhack\coursework\group_work\group_project_week5_6\second_project\data\raw\df_final_web_data_pt_1.txt')
df_3 = pd.read_csv(r'C:\Users\Cecilia\Downloads\ironhack\coursework\group_work\group_project_week5_6\second_project\data\raw\df_final_web_data_pt_2.txt')
merged_df = pd.concat([df_2, df_3], axis=0)

df_merged = df_1.merge(merged_df, on='client_id', how='inner')
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])
df_merged

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gender,num_accts,balance,calls_6_mnth,logons_6_mnth,variation,visitor_id,visit_id,process_step,date_time
0,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:51:13
1,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:47:50
2,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:46:45
3,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,step_3,2017-04-02 11:23:08
4,836976,6.0,73.0,60.5,Unknown,2.0,45105.30,6.0,9.0,Test,427070339_1413275162,228976764_46825473280_96584,step_2,2017-04-02 11:22:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321182,7468138,18.0,222.0,61.0,Female,3.0,209278.15,0.0,3.0,Test,317123960_46820919455,769876461_30381166055_830233,step_2,2017-03-30 23:59:15
321183,7468138,18.0,222.0,61.0,Female,3.0,209278.15,0.0,3.0,Test,317123960_46820919455,769876461_30381166055_830233,step_1,2017-03-30 23:58:51
321184,7468138,18.0,222.0,61.0,Female,3.0,209278.15,0.0,3.0,Test,317123960_46820919455,769876461_30381166055_830233,start,2017-03-30 23:58:40
321185,7468138,18.0,222.0,61.0,Female,3.0,209278.15,0.0,3.0,Test,317123960_46820919455,769876461_30381166055_830233,start,2017-03-30 23:55:11


In [27]:
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

#Sort control group
control_group_sorted = control_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

# Sort test group
test_group_sorted = test_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

In [28]:
import pandas as pd

# Define the bins and labels for age groups
bins = [0, 30, 40, 50, 100]  # These bins categorize ages into 4 groups
labels = ['Under 30', '30-39', '40-49', '50 and above']  # Age group labels

# Apply the bins to categorize the 'clnt_age' column
df_merged['age_group'] = pd.cut(df_merged['clnt_age'], bins=bins, labels=labels)

# Display the first few rows of the DataFrame to check the new 'age_group' column
print(df_merged[['clnt_age', 'age_group']].head())

   clnt_age     age_group
0      60.5  50 and above
1      60.5  50 and above
2      60.5  50 and above
3      60.5  50 and above
4      60.5  50 and above


In [29]:
# Function to get latest starts
def filter_latest_starts(group_df):
    starts_only = group_df[group_df['process_step'] == 'start']
    latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]
    # Merge back to get the full process after the last start
    return df_merged.merge(latest_starts[['client_id', 'date_time']], on=['client_id', 'date_time'], how='inner')

# Apply to both groups
filtered_control = filter_latest_starts(control_group)
filtered_test = filter_latest_starts(test_group)

#Check if it works
client_total_entries = df_merged[df_merged["client_id"] == 2304905]
display(client_total_entries)
client_last_start = filtered_control[filtered_control['client_id'] == 2304905 ]
display(client_last_start)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gender,num_accts,balance,calls_6_mnth,logons_6_mnth,variation,visitor_id,visit_id,process_step,date_time,age_group
11,2304905,7.0,94.0,58.0,Unknown,2.0,110860.3,6.0,9.0,Control,336210275_56978226960,724930510_65323235593_692754,confirm,2017-05-21 11:07:16,50 and above
12,2304905,7.0,94.0,58.0,Unknown,2.0,110860.3,6.0,9.0,Control,336210275_56978226960,724930510_65323235593_692754,step_3,2017-05-21 11:04:53,50 and above
13,2304905,7.0,94.0,58.0,Unknown,2.0,110860.3,6.0,9.0,Control,336210275_56978226960,724930510_65323235593_692754,step_2,2017-05-21 11:04:04,50 and above
14,2304905,7.0,94.0,58.0,Unknown,2.0,110860.3,6.0,9.0,Control,336210275_56978226960,724930510_65323235593_692754,step_1,2017-05-21 11:03:31,50 and above
15,2304905,7.0,94.0,58.0,Unknown,2.0,110860.3,6.0,9.0,Control,336210275_56978226960,724930510_65323235593_692754,start,2017-05-21 11:03:25,50 and above
16,2304905,7.0,94.0,58.0,Unknown,2.0,110860.3,6.0,9.0,Control,336210275_56978226960,724930510_65323235593_692754,start,2017-05-21 11:02:21,50 and above


,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gender,num_accts,balance,calls_6_mnth,logons_6_mnth,variation,visitor_id,visit_id,process_step,date_time,age_group
0,2304905,7.0,94.0,58.0,Unknown,2.0,110860.3,6.0,9.0,Control,336210275_56978226960,724930510_65323235593_692754,start,2017-05-21 11:03:25,50 and above


**Overall Stats**

In [30]:
# filter to get the latest start for each client
starts_only = df_merged[df_merged['process_step'] == 'start']
latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]

# filter to get the last confirmation for each client
confirmation_only = df_merged[df_merged['process_step'] == 'confirm']
latest_confirms = confirmation_only.loc[confirmation_only.groupby('client_id')['date_time'].idxmax()]

# merge to have both latest start and confirm per client
# Confirming datetime columns are of Timestamp type in merged DataFrame
latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))

# calculate process duration for those who completed the process
latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']

# Scalculate the average duration and compare with mode and mean
print("Average duration:", latest_start_confirms['process_duration'].mean())
print("Duration mode:", latest_start_confirms['process_duration'].mode())
print("Duration median:", latest_start_confirms['process_duration'].median())

Average duration: 0 days 22:42:16.093893650
Duration mode: 0   0 days 00:02:16
Name: process_duration, dtype: timedelta64[ns]
Duration median: 0 days 00:03:48


In [31]:
import pandas as pd

# Ensure 'date_time' is of datetime type
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'], errors='coerce')  # Coerce invalid formats to NaT

# Function to calculate process duration and statistics with outlier removal
def calculate_duration_stats(group_df):
    # Filter to get the latest start for each client
    starts_only = group_df[group_df['process_step'] == 'start']
    latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]

    # Filter to get the last confirmation for each client
    confirmation_only = group_df[group_df['process_step'] == 'confirm']
    latest_confirms = confirmation_only.loc[confirmation_only.groupby('client_id')['date_time'].idxmax()]

    # Merge to have both latest start and confirm per client
    latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))

    # Calculate process duration for those who completed the process
    latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']

    # Drop rows with NaT values in process_duration
    latest_start_confirms = latest_start_confirms.dropna(subset=['process_duration'])

    # Convert process_duration to numeric (seconds) for statistical calculations
    latest_start_confirms['process_duration_seconds'] = latest_start_confirms['process_duration'].dt.total_seconds()

    # Step 1: Detect outliers using IQR
    Q1 = latest_start_confirms['process_duration_seconds'].quantile(0.25)
    Q3 = latest_start_confirms['process_duration_seconds'].quantile(0.75)
    IQR = Q3 - Q1

    # Define the outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Step 2: Filter out the outliers
    filtered_df = latest_start_confirms[(latest_start_confirms['process_duration_seconds'] >= lower_bound) & 
                                        (latest_start_confirms['process_duration_seconds'] <= upper_bound)]

    # Step 3: Calculate the average, mode, and median durations
    avg_duration = pd.to_timedelta(filtered_df['process_duration_seconds'].mean(), unit='s')
    
    # Check if mode is non-empty before accessing the first element
    mode_duration = filtered_df['process_duration_seconds'].mode()
    if not mode_duration.empty:
        mode_duration = pd.to_timedelta(mode_duration[0], unit='s')
    else:
        mode_duration = pd.to_timedelta(0, unit='s')  # Default to zero if no mode is found
    
    median_duration = pd.to_timedelta(filtered_df['process_duration_seconds'].median(), unit='s')

    return avg_duration, mode_duration, median_duration

# Calculate statistics for the entire dataset (with outlier removal)
avg_duration, mode_duration, median_duration = calculate_duration_stats(df_merged)

# Print results
print("Overall Statistics (With Outliers Removed):")
print(f"Average duration: {avg_duration}")
print(f"Duration mode: {mode_duration}")
print(f"Duration median: {median_duration}")

Overall Statistics (With Outliers Removed):
Average duration: 0 days 00:04:18.338016588
Duration mode: 0 days 00:02:16
Duration median: 0 days 00:03:33


**Test and Control Group Stats**

In [32]:
import pandas as pd

# Ensure 'date_time' is of datetime type for both start and confirm stages
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'], errors='coerce')  # Coerce invalid formats to NaT

# Check the columns in df_merged to verify the structure
print(df_merged.columns)

# Split the dataset into control and test groups using the 'variation' column
control_group = df_merged[df_merged['variation'] == 'control']
test_group = df_merged[df_merged['variation'] == 'test']

# Function to calculate process duration and statistics
def calculate_duration_stats(group_df):
    # Filter to get the latest start for each client
    starts_only = group_df[group_df['process_step'] == 'start']
    latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]

    # Filter to get the last confirmation for each client
    confirmation_only = group_df[group_df['process_step'] == 'confirm']
    latest_confirms = confirmation_only.loc[confirmation_only.groupby('client_id')['date_time'].idxmax()]

    # Merge to have both latest start and confirm per client
    latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))

    # Calculate process duration for those who completed the process
    latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']

    # Drop rows with NaT values in process_duration
    latest_start_confirms = latest_start_confirms.dropna(subset=['process_duration'])

    # Convert process_duration to numeric (seconds) for statistical calculations
    latest_start_confirms['process_duration_seconds'] = latest_start_confirms['process_duration'].dt.total_seconds()

    # Calculate average, mode, and median durations
    avg_duration = pd.to_timedelta(latest_start_confirms['process_duration_seconds'].mean(), unit='s')
    
    # Check if mode is non-empty before accessing the first element
    mode_duration = latest_start_confirms['process_duration_seconds'].mode()
    if not mode_duration.empty:
        mode_duration = pd.to_timedelta(mode_duration[0], unit='s')
    else:
        mode_duration = pd.to_timedelta(0, unit='s')  # Default to zero if no mode is found
    
    median_duration = pd.to_timedelta(latest_start_confirms['process_duration_seconds'].median(), unit='s')

    return avg_duration, mode_duration, median_duration

# Calculate statistics for control group
control_avg_duration, control_mode_duration, control_median_duration = calculate_duration_stats(control_group)

# Calculate statistics for test group
test_avg_duration, test_mode_duration, test_median_duration = calculate_duration_stats(test_group)

# Print results for control group
print("Control Group Statistics:")
print(f"Average duration: {control_avg_duration}")
print(f"Duration mode: {control_mode_duration}")
print(f"Duration median: {control_median_duration}")
print("\n")  # Add a newline for better separation

# Print results for test group
print("Test Group Statistics:")
print(f"Average duration: {test_avg_duration}")
print(f"Duration mode: {test_mode_duration}")
print(f"Duration median: {test_median_duration}")

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gender',
       'num_accts', 'balance', 'calls_6_mnth', 'logons_6_mnth', 'variation',
       'visitor_id', 'visit_id', 'process_step', 'date_time', 'age_group'],
      dtype='object')
Control Group Statistics:
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT


Test Group Statistics:
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT


In [33]:
import pandas as pd

# Ensure 'date_time' is of datetime type
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'], errors='coerce')  # Coerce invalid formats to NaT

# Function to calculate process duration and statistics with outlier removal for each group
def calculate_duration_stats(group_df):
    # Filter to get the latest start for each client
    starts_only = group_df[group_df['process_step'] == 'start']
    latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]

    # Filter to get the last confirmation for each client
    confirmation_only = group_df[group_df['process_step'] == 'confirm']
    latest_confirms = confirmation_only.loc[confirmation_only.groupby('client_id')['date_time'].idxmax()]

    # Merge to have both latest start and confirm per client
    latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))

    # Calculate process duration for those who completed the process
    latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']

    # Drop rows with NaT values in process_duration
    latest_start_confirms = latest_start_confirms.dropna(subset=['process_duration'])

    # Convert process_duration to numeric (seconds) for statistical calculations
    latest_start_confirms['process_duration_seconds'] = latest_start_confirms['process_duration'].dt.total_seconds()

    # Step 1: Detect outliers using IQR
    Q1 = latest_start_confirms['process_duration_seconds'].quantile(0.25)
    Q3 = latest_start_confirms['process_duration_seconds'].quantile(0.75)
    IQR = Q3 - Q1

    # Define the outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Step 2: Filter out the outliers
    filtered_df = latest_start_confirms[(latest_start_confirms['process_duration_seconds'] >= lower_bound) & 
                                        (latest_start_confirms['process_duration_seconds'] <= upper_bound)]

    # Step 3: Calculate the average, mode, and median durations
    avg_duration = pd.to_timedelta(filtered_df['process_duration_seconds'].mean(), unit='s')
    
    # Check if mode is non-empty before accessing the first element
    mode_duration = filtered_df['process_duration_seconds'].mode()
    if not mode_duration.empty:
        mode_duration = pd.to_timedelta(mode_duration[0], unit='s')
    else:
        mode_duration = pd.to_timedelta(0, unit='s')  # Default to zero if no mode is found
    
    median_duration = pd.to_timedelta(filtered_df['process_duration_seconds'].median(), unit='s')

    return avg_duration, mode_duration, median_duration

# Split the dataset into control and test groups based on the 'variation' column
control_group = df_merged[df_merged['variation'] == 'control']
test_group = df_merged[df_merged['variation'] == 'test']

# Calculate statistics for control group
control_avg_duration, control_mode_duration, control_median_duration = calculate_duration_stats(control_group)

# Calculate statistics for test group
test_avg_duration, test_mode_duration, test_median_duration = calculate_duration_stats(test_group)

# Print results for control group
print("Control Group Statistics (With Outliers Removed):")
print(f"Average duration: {control_avg_duration}")
print(f"Duration mode: {control_mode_duration}")
print(f"Duration median: {control_median_duration}")

# Print results for test group
print("\nTest Group Statistics (With Outliers Removed):")
print(f"Average duration: {test_avg_duration}")
print(f"Duration mode: {test_mode_duration}")
print(f"Duration median: {test_median_duration}")

Control Group Statistics (With Outliers Removed):
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT

Test Group Statistics (With Outliers Removed):
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT


In [34]:
#completion rate based on visit_id
def calculate_within_visit_completion_rate(group):
    # Total unique visits that started
    started_visits = group[group['process_step'] == 'start']['visit_id'].nunique()

    # Unique visits that completed each step
    completed_visits = (
        group[group['process_step'] != 'start']
        .groupby('process_step')['visit_id']
        .nunique()
        .reset_index(name='completed_visits')
    )

    # Add the total started visits as a constant column
    completed_visits['started_visits'] = started_visits

    # Calculate the completion rate
    completed_visits['completion_rate'] = (
        completed_visits['completed_visits'] / completed_visits['started_visits']
    ) * 100

    return completed_visits

# Calculate completion rates for the control and test groups
control_completion_rate = calculate_within_visit_completion_rate(control_group_sorted)
test_completion_rate = calculate_within_visit_completion_rate(test_group_sorted)

# Display the results
print("Control Group Within-Visit Completion Rates:")
display(control_completion_rate)

print("Test Group Within-Visit Completion Rates:")
display(test_completion_rate)

Control Group Within-Visit Completion Rates:


,process_step,completed_visits,started_visits,completion_rate
0,confirm,16039,30903,51.901110
1,step_1,23541,30903,76.177070
2,step_2,20131,30903,65.142543
3,step_3,18293,30903,59.194900


Test Group Within-Visit Completion Rates:


,process_step,completed_visits,started_visits,completion_rate
0,confirm,21725,33142,65.551264
1,step_1,28273,33142,85.308672
2,step_2,24493,33142,73.903204
3,step_3,22180,33142,66.924145


In [39]:
import pandas as pd

# Sample DataFrame
data = {
    'process_step': ['confirm', 'start', 'step_1', 'step_2', 'step_3', 'confirm', 'start', 'step_1', 'step_2', 'step_3'],
    'date_time': pd.to_datetime([
        '2024-11-20 10:00:00', '2024-11-20 08:00:00', '2024-11-20 08:15:00', 
        '2024-11-20 09:00:00', '2024-11-20 09:30:00', '2024-11-21 10:00:00', 
        '2024-11-21 08:10:00', '2024-11-21 08:40:00', '2024-11-21 09:20:00', '2024-11-21 09:45:00'
    ]),
    'visit_id': [101, 101, 101, 101, 101, 102, 102, 102, 102, 102],
    'client_id': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
    'clnt_age': [25, 35, 42, 55, 29, 48, 33, 60, 19, 52],  # Client age
    'gender': ['Male', 'Female', 'Female', 'Male', 'Female', 'Male', 'Female', 'Unknown', 'Female', 'Male'],  # Client gender with "Unknown"
    'process_duration': [120, 150, 200, 250, 130, 180, 160, 270, 110, 240],  # Process duration
    'group': ['test', 'test', 'test', 'test', 'test', 'test', 'test', 'test', 'test', 'test'],  # Group
    'variation': ['Control', 'Control', 'Control', 'Control', 'Control', 'Test', 'Test', 'Test', 'Test', 'Test']  # Variation
}

# Create DataFrame
df_merged = pd.DataFrame(data)

# Handle "Unknown" gender by filling NaN or incorrect values with 'Unknown'
df_merged['gender'] = df_merged['gender'].fillna('Unknown').replace('Unknown', 'Unknown')

# Define age groups
bins = [0, 30, 40, 50, 100]
labels = ['Under 30', '30-39', '40-49', '50 and above']

# Create the 'age_group' column based on 'clnt_age'
df_merged['age_group'] = pd.cut(df_merged['clnt_age'], bins=bins, labels=labels, right=False)

# Sort the DataFrame by age and gender
df_merged_sorted = df_merged.sort_values(by=['clnt_age', 'gender'], ascending=[True, True])

# Filter control and test groups
control_group = df_merged_sorted[df_merged_sorted['variation'] == 'Control']
test_group = df_merged_sorted[df_merged_sorted['variation'] == 'Test']

# Sort control group
control_group_sorted = control_group.sort_values(by=['visit_id', 'process_step', 'date_time'])

# Sort test group
test_group_sorted = test_group.sort_values(by=['visit_id', 'process_step', 'date_time'])

# Completion rate based on visit_id
def calculate_within_visit_completion_rate(group):
    # Total unique visits that started
    started_visits = group[group['process_step'] == 'start']['visit_id'].nunique()

    # Unique visits that completed each step
    completed_visits = (
        group[group['process_step'] != 'start']
        .groupby('process_step')['visit_id']
        .nunique()
        .reset_index(name='completed_visits')
    )

    # Add the total started visits as a constant column
    completed_visits['started_visits'] = started_visits

    # Calculate the completion rate
    completed_visits['completion_rate'] = (
        completed_visits['completed_visits'] / completed_visits['started_visits']
    ) * 100

    return completed_visits

# Display completion rates for each age group and gender, including "Unknown" gender
for age_group_label in df_merged_sorted['age_group'].unique():
    print(f"\nCompletion Rates for Age Group: {age_group_label}")
    
    for gender_label in df_merged_sorted['gender'].unique():
        print(f"  Gender: {gender_label}")
        
        # Filter for control group by age group and gender
        control_group_filtered = control_group_sorted[
            (control_group_sorted['age_group'] == age_group_label) & 
            (control_group_sorted['gender'] == gender_label)
        ]
        control_completion_rate = calculate_within_visit_completion_rate(control_group_filtered)
        print(f"    Control Group Completion Rates:\n", control_completion_rate)
        
        # Filter for test group by age group and gender
        test_group_filtered = test_group_sorted[
            (test_group_sorted['age_group'] == age_group_label) & 
            (test_group_sorted['gender'] == gender_label)
        ]
        test_completion_rate = calculate_within_visit_completion_rate(test_group_filtered)
        print(f"    Test Group Completion Rates:\n", test_completion_rate)


Completion Rates for Age Group: Under 30
  Gender: Female
    Control Group Completion Rates:
   process_step  completed_visits  started_visits  completion_rate
0       step_3                 1               0              inf
    Test Group Completion Rates:
   process_step  completed_visits  started_visits  completion_rate
0       step_2                 1               0              inf
  Gender: Male
    Control Group Completion Rates:
   process_step  completed_visits  started_visits  completion_rate
0      confirm                 1               0              inf
    Test Group Completion Rates:
 Empty DataFrame
Columns: [process_step, completed_visits, started_visits, completion_rate]
Index: []
  Gender: Unknown
    Control Group Completion Rates:
 Empty DataFrame
Columns: [process_step, completed_visits, started_visits, completion_rate]
Index: []
    Test Group Completion Rates:
 Empty DataFrame
Columns: [process_step, completed_visits, started_visits, completion_rate]
Index:

In [41]:
import pandas as pd

# Ensure 'date_time' is of datetime type
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'], errors='coerce')  # Coerce invalid formats to NaT

# Create 'age_group' from 'clnt_age'
bins = [0, 30, 40, 50, 100]
labels = ['Under 30', '30-39', '40-49', '50 and above']
df_merged['age_group'] = pd.cut(df_merged['clnt_age'], bins=bins, labels=labels, right=False)

# Handle "Unknown" gender
df_merged['gender'] = df_merged['gender'].fillna('Unknown')

# Function to calculate process duration and statistics with outlier removal for each group
def calculate_duration_stats(group_df):
    # Filter to get the latest start for each client
    starts_only = group_df[group_df['process_step'] == 'start']
    latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]

    # Filter to get the last confirmation for each client
    confirmation_only = group_df[group_df['process_step'] == 'confirm']
    latest_confirms = confirmation_only.loc[confirmation_only.groupby('client_id')['date_time'].idxmax()]

    # Merge to have both latest start and confirm per client
    latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))

    # Calculate process duration for those who completed the process
    latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']

    # Drop rows with NaT values in process_duration
    latest_start_confirms = latest_start_confirms.dropna(subset=['process_duration'])

    # Convert process_duration to numeric (seconds) for statistical calculations
    latest_start_confirms['process_duration_seconds'] = latest_start_confirms['process_duration'].dt.total_seconds()

    # Step 1: Detect outliers using IQR
    Q1 = latest_start_confirms['process_duration_seconds'].quantile(0.25)
    Q3 = latest_start_confirms['process_duration_seconds'].quantile(0.75)
    IQR = Q3 - Q1

    # Define the outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Step 2: Filter out the outliers
    filtered_df = latest_start_confirms[(latest_start_confirms['process_duration_seconds'] >= lower_bound) & 
                                        (latest_start_confirms['process_duration_seconds'] <= upper_bound)]

    # Step 3: Calculate the average, mode, and median durations
    avg_duration = pd.to_timedelta(filtered_df['process_duration_seconds'].mean(), unit='s')
    
    # Check if mode is non-empty before accessing the first element
    mode_duration = filtered_df['process_duration_seconds'].mode()
    if not mode_duration.empty:
        mode_duration = pd.to_timedelta(mode_duration[0], unit='s')
    else:
        mode_duration = pd.to_timedelta(0, unit='s')  # Default to zero if no mode is found
    
    median_duration = pd.to_timedelta(filtered_df['process_duration_seconds'].median(), unit='s')

    return avg_duration, mode_duration, median_duration

# Split the dataset into control and test groups based on the 'variation' column
control_group = df_merged[df_merged['variation'] == 'control']
test_group = df_merged[df_merged['variation'] == 'test']

# Loop through the unique combinations of age group and gender
for age_group_label in df_merged['age_group'].unique():
    for gender_label in df_merged['gender'].unique():
        
        print(f"\nAge Group: {age_group_label}, Gender: {gender_label}")
        
        # Filter control group by age group and gender
        control_group_filtered = control_group[(control_group['age_group'] == age_group_label) & 
                                               (control_group['gender'] == gender_label)]
        # Calculate statistics for control group
        control_avg_duration, control_mode_duration, control_median_duration = calculate_duration_stats(control_group_filtered)

        print(f"Control Group Statistics (With Outliers Removed):")
        print(f"Average duration: {control_avg_duration}")
        print(f"Duration mode: {control_mode_duration}")
        print(f"Duration median: {control_median_duration}")

        # Filter test group by age group and gender
        test_group_filtered = test_group[(test_group['age_group'] == age_group_label) & 
                                         (test_group['gender'] == gender_label)]
        # Calculate statistics for test group
        test_avg_duration, test_mode_duration, test_median_duration = calculate_duration_stats(test_group_filtered)

        print(f"\nTest Group Statistics (With Outliers Removed):")
        print(f"Average duration: {test_avg_duration}")
        print(f"Duration mode: {test_mode_duration}")
        print(f"Duration median: {test_median_duration}")


Age Group: Under 30, Gender: Male
Control Group Statistics (With Outliers Removed):
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT

Test Group Statistics (With Outliers Removed):
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT

Age Group: Under 30, Gender: Female
Control Group Statistics (With Outliers Removed):
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT

Test Group Statistics (With Outliers Removed):
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT

Age Group: Under 30, Gender: Unknown
Control Group Statistics (With Outliers Removed):
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT

Test Group Statistics (With Outliers Removed):
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: NaT

Age Group: 30-39, Gender: Male
Control Group Statistics (With Outliers Removed):
Average duration: NaT
Duration mode: 0 days 00:00:00
Duration median: 

In [44]:
import pandas as pd

# Ensure 'date_time' is of datetime type
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'], errors='coerce')  # Coerce invalid formats to NaT

# Create 'age_group' from 'clnt_age' (if not already done)
bins = [0, 30, 40, 50, 100]
labels = ['Under 30', '30-39', '40-49', '50 and above']
df_merged['age_group'] = pd.cut(df_merged['clnt_age'], bins=bins, labels=labels, right=False)

# Handle "Unknown" gender (if not already done)
df_merged['gender'] = df_merged['gender'].fillna('Unknown')

# Function to calculate and display process duration statistics with age and gender filters
def calculate_and_display_stats(group_df, age_group_label, gender_label):
    # Filter control group and test group by age group and gender
    group_filtered = group_df[(group_df['age_group'] == age_group_label) & 
                               (group_df['gender'] == gender_label)]
    
    # Filter to get the latest start for each client
    starts_only = group_filtered[group_filtered['process_step'] == 'start']
    latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]

    # Filter to get the last confirmation for each client
    confirmation_only = group_filtered[group_filtered['process_step'] == 'confirm']
    latest_confirms = confirmation_only.loc[confirmation_only.groupby('client_id')['date_time'].idxmax()]

    # Merge to have both latest start and confirm per client
    latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))

    # Calculate process duration for those who completed the process
    latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']

    # Calculate the average, mode, and median durations
    avg_duration = latest_start_confirms['process_duration'].mean()
    mode_duration = latest_start_confirms['process_duration'].mode()
    median_duration = latest_start_confirms['process_duration'].median()

    # Display the results
    print(f"Statistics for Age Group: {age_group_label}, Gender: {gender_label}")
    print(f"Average duration: {avg_duration}")
    print(f"Duration mode: {mode_duration[0] if not mode_duration.empty else 'N/A'}")
    print(f"Duration median: {median_duration}")

# Example: Loop through all combinations of age groups and genders
for age_group_label in df_merged['age_group'].unique():
    for gender_label in df_merged['gender'].unique():
        # Call the function for each age group and gender
        calculate_and_display_stats(df_merged, age_group_label, gender_label)

Statistics for Age Group: Under 30, Gender: Male
Average duration: NaT
Duration mode: N/A
Duration median: NaT
Statistics for Age Group: Under 30, Gender: Female
Average duration: NaT
Duration mode: N/A
Duration median: NaT
Statistics for Age Group: Under 30, Gender: Unknown
Average duration: NaT
Duration mode: N/A
Duration median: NaT
Statistics for Age Group: 30-39, Gender: Male
Average duration: NaT
Duration mode: N/A
Duration median: NaT
Statistics for Age Group: 30-39, Gender: Female
Average duration: NaT
Duration mode: N/A
Duration median: NaT
Statistics for Age Group: 30-39, Gender: Unknown
Average duration: NaT
Duration mode: N/A
Duration median: NaT
Statistics for Age Group: 40-49, Gender: Male
Average duration: NaT
Duration mode: N/A
Duration median: NaT
Statistics for Age Group: 40-49, Gender: Female
Average duration: NaT
Duration mode: N/A
Duration median: NaT
Statistics for Age Group: 40-49, Gender: Unknown
Average duration: NaT
Duration mode: N/A
Duration median: NaT
Stat